# Lab Recap

Can we retrieve 3 random facts from the [Internet Chuck Norris Database](http://www.icndb.com/api/) and display them as formatted HTML?

# Sorting

Sorting algorithms are one of the most frequently studied class of algorithms. Not only are there many such algorithms, it is also very easy to implement them incorrectly.

Can we implement [Bubble sort](https://en.wikipedia.org/wiki/Bubble_sort) in Python? How can we test it?

See `sorting.py`.

Can we also implement [merge sort](https://en.wikipedia.org/wiki/Merge_sort)?

See `sorting.py`.

# Error Handling

What do we do when things go wrong?

## Using `None`

Consider a function `find(needle, haystack)` that finds the position of `needle` in the list `haystack`. Can we implement this function?

Does python have built-in functionality for doing this?

## Exceptions

Can we change our `find()` function so that it raises an exception?

Consider the function, `score(card_points, cards)`, from the week 4 lab. It takes in a dictionary representing point values for different cards and a list of cards, and calculates the total score for that list.

Can we implement this function differently using exceptions?

A number, `x`, inside a list, `list`, is said to lead back to itself in 1 step if `list[x] == x`. A number leads back to itself in 2 steps if `list[list[x]] == x`. Which of the numbers in the following list lead back to themselves in 3 steps.

In [ ]:
list = [0, 2, 1, 4, 5, 3, 9, 6, 7]

# Alternative ways of working with data structures

Can we implement the `score(card_points, cards)` function without conditionals or exception handling?

## List/Dictionary Comprehensions

Can we extract a list of zids from these email addresses?

In [ ]:
emails = ["z1234567@student.unsw.edu.au", "z7654321@unsw.edu.au", "z7891234@ad.unsw.edu.au", "z1357924@student.unsw.edu.au"]

Can we create a dictionary with the zids as keys and the email addresses as values?

Find all the words that are palindromes in this list:

In [ ]:
words = ["kayak", "hello", "racecar", "madam", "moon", "noon", "shish", "level"]

# Functional Programming

Functional programming is a programming paradigm where programs are built by composing *pure* functions

## Functions as values



Can we create a list of functions?

Can we write the function `apply_list(functions, value)` that applies all the functions in the list `functions` to `value` and returns the results in a new list?

Can we do the same with a single function and a list of values?

Do functions always have to be given a name?

## List Combinators

List combinators are functions that perform operations on lists, typically used as an alternative to loops.

Many are available form `functools` and `itertools`.

Consider a function, `common_letters(words)`, that calculates $2a + e$ where $a$ is the number of times the letter "a" appears in all the words in `words` and $e$ is the number of times the letter "e" occurs. Can we write such a function without using any form of loop.

Consider this dictionary of first year courses and the number of enrolments they have:

In [ ]:
first_year_courses = { "COMP1010": 45, "COMP1511": 560, "COMP1911": 100, "MATH1131": 1034, "MATH1231": 895, "FINS1612": 423, "FINS1613": 587, "ACCT1501": 327 }

Can we get a list of all the courses with 500 or more enrolments?

Can we construct a dictionary containing all the subjects areas and the number of enrolments in those subject areas?

# Code Golf

Code golf is the challenge of solving programming problems with the least amount of code possible. While a fun exercise, this is not necessarily a good way to write code.

Some tips are [here](https://www.geeksforgeeks.org/code-golfing-in-python/).

# 